### Heroes Of Pymoli Data Analysis

-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,101,Final Critic,4.19


## Player Count

* Display the total number of players


In [2]:
pd.DataFrame([len(purchase_data['SN'].unique())], index=None, columns=['Total Players'])

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
cols=['Number of Unique Items', 'Average Price','Number of Purchases', 'Total Revenue']
rowdata = []
rowdata.append(len(purchase_data["Item ID"].unique()))
rowdata.append(purchase_data["Price"].mean())

rowdata.append(purchase_data["Purchase ID"].count())
rowdata.append(purchase_data["Price"].sum())
rowdata
analysis = pd.DataFrame( np.array([rowdata]), index=None, columns=cols)
analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183.0,3.050987,780.0,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# get all the unique user names first

uniqueSN =  pd.Series(purchase_data['SN'].unique()) 
uniqueSN = uniqueSN.rename("SN")
uniqueSNdf = pd.DataFrame(uniqueSN)
uniqueSNdf

,SN
0,Lisim78
1,Lisovynya38
2,Ithergue48
3,Chamassasya86
4,Iskosia90
...,...
571,Hala31
572,Jiskjask80
573,Aethedru70
574,Yathecal72


In [5]:
# inner join unique names with genders
inner_merge_df = pd.merge(uniqueSNdf, purchase_data, on="SN", how="left")
inner_merge_df = inner_merge_df.drop_duplicates(subset=['SN'])
gender_df = inner_merge_df[['SN','Gender']]

#pull out number of people of each gender.
numberTotalPeople = gender_df['Gender'].count()
#filter Males, Females, etc. 
numberMale = gender_df[gender_df['Gender']=='Male'].count()[0]
numberFemale = gender_df[gender_df['Gender']=='Female'].count()[0]
numberOtherUndisclosed = gender_df[gender_df['Gender']=='Other / Non-Disclosed'].count()[0]
numberTotalPeople

576

In [6]:

genderGroupSeries = gender_df.groupby("Gender").count()['SN']
#df.rename(columns={"A": "a", "B": "c"})
genderGroupSeries = genderGroupSeries.rename("Total People")

#
countOfGenders = [numberMale,numberFemale, numberOtherUndisclosed]
percentages = countOfGenders / numberTotalPeople * 100
percentages


array([84.02777778, 14.0625    ,  1.90972222])

In [7]:


#genderGroupDF['Percentage of Players']=genderGroupDF / totalPeople
#float( genderGroupDF['Total Count'])/ totalPeople
genderAnalysisDF = pd.DataFrame(genderGroupSeries)
genderAnalysisDF['percentage of players'] = percentages
genderAnalysisDF

#purchase_data['SN', 'Gender'].unique()

,Total People,percentage of players
Gender,,
Female,81,84.027778
Male,484,14.062500
Other / Non-Disclosed,11,1.909722



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
# For reference: 	Purchase Count 	Average Purchase Price 	Total Purchase Value 	Avg Total Purchase per Person


# take a group and calculate various sums/means

genderGroupDF = purchase_data.groupby("Gender")
totalPurchaseCountByGender = genderGroupDF.count()['Purchase ID']
averagePriceByGender = genderGroupDF.mean()['Price']
totalPricesByGender = genderGroupDF.sum()['Price']

# use prior DF that counted total people
averagePricebyGenderPerPerson = totalPricesByGender  / genderAnalysisDF['Total People']

#load up the new DF
genderSummaryDF = pd.DataFrame() 
genderSummaryDF['Purchase Count'] = totalPurchaseCountByGender
genderSummaryDF['Average Purchase Price'] = averagePriceByGender
genderSummaryDF['Total Purchase Value'] = totalPricesByGender
genderSummaryDF['Avg Total Purchase per Person'] = averagePricebyGenderPerPerson
genderSummaryDF

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [9]:
# Create bins and bin labels for the Age column

age_bins = [0,9,14,19,24,29,34,39,200]
age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34" , "35-40", "40+"]

# Bin the Age column
# cut() returns a Pandas Series containing each of the binned column's values translated into their corresponding bins

ageBinsSeries = pd.cut(purchase_data["Age"], age_bins, labels=age_labels)
ageBinsSeries.rename("Age")
ageBinsSeries

0      20-24
1        40+
2      20-24
3      20-24
4      20-24
       ...  
775    20-24
776    20-24
777    20-24
778      <10
779    20-24
Name: Age, Length: 780, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-40 < 40+]

In [10]:
ageBinDF = purchase_data
ageBinDF["Age Bin"] = ageBinsSeries
ageBinDF

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bin
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,101,Final Critic,4.19,<10


In [11]:
ageBinDFUnique = ageBinDF.drop_duplicates(subset=['SN'])
ageBinDFUnique

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bin
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
773,773,Hala31,21,Male,19,"Pursuit, Cudgel of Necromancy",1.02,20-24
774,774,Jiskjask80,11,Male,101,Final Critic,4.19,10-14
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24


In [12]:
summaryAgeDF=pd.DataFrame()
ageBinCount = []
filteredseries   = ageBinDFUnique[ageBinDFUnique["Age Bin"]=="<10"]
# gender_df[gender_df['Gender']=='Male']
for label in age_labels:
    ageBinCount.append(ageBinDFUnique[ageBinDFUnique["Age Bin"]==label].count()[0])
#ageBinsSeries[ageBinsSeries['Age'==label]].count()
    #ages.append(ageBinsSeries[ageBinsSeries['Age'==label]].count())
#summaryAgeDF['Total Count'] = ages
ageBinCount

[17, 22, 107, 258, 77, 52, 31, 12]

In [13]:
ageBinIndex  = ageBinDFUnique['Age Bin'].drop_duplicates()
ageBinIndex = ageBinIndex.sort_values()
# vehicles_df.sort_values("Fuel Economy (mpg)")
summaryAgeDF=pd.DataFrame(index=ageBinIndex)
#summaryAgeDF
#summaryAgeDF['Total Count'] = ageBinCount
#summaryAgeDF= summaryAgeDF.drop_duplicates(subset=['Age Bin'])
summaryAgeDF['Total Count'] = ageBinCount
summaryAgeDF['Percentage of Players'] = ageBinCount / numberTotalPeople * 100
summaryAgeDF

,Total Count,Percentage of Players
Age Bin,,
<10,17,2.951389
10-14,22,3.819444
15-19,107,18.576389
20-24,258,44.791667
25-29,77,13.368056
30-34,52,9.027778
35-40,31,5.381944
40+,12,2.083333


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [14]:
#summaryAgeDF['Average Purchase Price'] = ageBinDF['Price'].sum()
averagePurchasePriceList =[]
for label in age_labels:
    print(label)
    averagePurchasePriceList.append(ageBinDF[ageBinDF["Age Bin"]==label].mean()['Price'])
summaryAgeDF['Average Purchase Price'] = averagePurchasePriceList
averagePurchasePriceList



<10
10-14
15-19
20-24
25-29
30-34
35-40
40+


[3.353478260869565,
 2.956428571428571,
 3.03595588235294,
 3.0522191780821935,
 2.900990099009899,
 2.9315068493150687,
 3.6017073170731706,
 2.9415384615384617]

In [16]:
totalSpentByAge = []
for label in age_labels:
    print(label)
    totalSpentByAge.append(ageBinDF[ageBinDF["Age Bin"]==label].sum()['Price'])

summaryAgeDF['Total Purchase Value'] = totalSpentByAge
#summaryAgeDF['Avg Total Purchase Per Person'] = ageBinCount

summaryAgeDF['Average Purchase Value Per Person'] = np.array(totalSpentByAge) / np.array(ageBinCount)
summaryAgeDF

<10
10-14
15-19
20-24
25-29
30-34
35-40
40+


,Total Count,Percentage of Players,Average Purchase Price,Total Purchase Value,Average Purchase Value Per Person
Age Bin,,,,,
<10,17,2.951389,3.353478,77.13,4.537059
10-14,22,3.819444,2.956429,82.78,3.762727
15-19,107,18.576389,3.035956,412.89,3.858785
20-24,258,44.791667,3.052219,1114.06,4.318062
25-29,77,13.368056,2.900990,293.00,3.805195
30-34,52,9.027778,2.931507,214.00,4.115385
35-40,31,5.381944,3.601707,147.67,4.763548
40+,12,2.083333,2.941538,38.24,3.186667


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [37]:
topSpenders = ageBinDF.groupby("SN").sum().sort_values('Price', ascending=False)
#df.rename(columns={"A": "a", "B": "b", "C": "c"}, errors="raise")
topSpenders = topSpenders.rename(columns = {'Price':'Total Spent'} , errors="raise") 
topSpenders['Average Price'] = topSpenders['Total Spent'] / ageBinDF.groupby("SN").count()['Purchase ID']
topSpenders['Purchase Count'] = ageBinDF.groupby("SN").count()['Purchase ID']

topSpenders
#topSpenders[:,'Price':]
#ageBinDF.groupby("SN").count().sort_values('Price')


topSpenders.loc[:, 'Total Spent' :]

,Total Spent,Average Price,Purchase Count
SN,,,
Lisosia93,18.96,3.792000,5
Idastidru52,15.45,3.862500,4
Chamjask73,13.83,4.610000,3
Iral74,13.62,3.405000,4
Iskadarya95,13.10,4.366667,3
...,...,...,...
Ililsasya43,1.02,1.020000,1
Irilis75,1.02,1.020000,1
Aidai61,1.01,1.010000,1


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [59]:
#grouped_speed_vehicles_df = vehicles_df.groupby("Speed")
#grouped_speed_vehicles_df[["Horsepower", "Torque (lb-ft)"]].mean()

#.count().sort_values('Item ID', ascending=False)
topSellers = pd.DataFrame() 

ageBinDF.groupby("Item ID").count()['Purchase ID']
#mostPopularItemsGroup[['Item ID']].count()

topSellers['Amount Sold'] =  ageBinDF.groupby("Item ID").count()['Purchase ID']
topSellers['Item Price'] =  ageBinDF.groupby("Item Name").count()['Price']
topSellers.sort_values('Amount Sold', ascending=False)

#df.rename(columns={"A": "a", "B": "b", "C": "c"}, errors="raise")
## mostPopularItems = mostPopularItems.rename(columns = {'Item ID':'Total Purchased'} , errors="raise") 
#topSpenders['Average Price'] = topSpenders['Total Spent'] / ageBinDF.groupby("SN").count()['Purchase ID']
#topSpenders['Purchase Count'] = ageBinDF.groupby("SN").count()['Purchase ID']

## mostPopularItems
#topSpenders[:,'Price':]
#ageBinDF.groupby("SN").count().sort_values('Price')


## mostPopularItems.loc[:, 'Total Purchased' :]

,Amount Sold,Item Price
Item ID,,
178,12,NaN
145,9,NaN
108,9,NaN
82,9,NaN
19,8,NaN
...,...,...
104,1,NaN
23,1,NaN
180,1,NaN


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

